<a href="https://colab.research.google.com/github/sayandeepmondal/Credit-Card-Fraud-Detection/blob/main/CreditCardFraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
from sklearn.exceptions import ConvergenceWarning

In [2]:
#loading dataset to pandas dataframe
credit_card_data=pd.read_csv('/content/creditcard.csv')

In [3]:
#print first 5 rows of dataframe
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [4]:
#print last 5 rows of dataframe
credit_card_data.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
43659,41598,-0.538686,-0.635328,-0.041569,-3.022158,0.771776,3.556774,-1.418820,1.371999,-2.475053,0.765158,-0.599911,-1.157990,0.590581,-0.141037,0.714276,0.236656,0.044727,0.834430,0.173313,-0.103447,-0.090409,-0.310254,0.168510,0.955413,-0.427495,-0.375891,0.051010,0.072416,60.01,0.0
43660,41599,-0.349615,-2.860571,0.297766,0.801212,-1.523995,1.164260,0.037484,0.325220,0.862825,-0.636598,1.256949,1.158141,-0.562951,-0.026724,-0.388570,-0.353574,0.489822,-0.829914,-0.270441,1.330992,0.315370,-0.503099,-0.514747,-0.193890,-0.327091,0.875820,-0.144908,0.125368,745.56,0.0
43661,41599,-3.001222,2.899766,0.726874,-0.729992,-0.312792,-0.735557,0.960944,-0.276141,2.128747,3.756951,0.604166,0.179630,0.105197,-1.115528,1.296973,-0.612735,-0.324341,-1.181626,-0.823525,1.737831,-0.645540,-0.613956,0.111663,0.338567,0.268357,0.076981,1.184716,0.491066,8.99,0.0
43662,41599,1.042342,-0.390001,-0.463680,-1.737393,0.450611,0.604354,0.068099,0.278030,0.555053,-1.074384,2.738063,1.966304,0.214837,0.824648,1.767730,-1.843145,0.935039,-2.181580,-0.266782,-0.104876,-0.137137,-0.286887,0.143336,-0.986810,0.135599,-0.185344,0.035590,-0.008304,48.00,0.0
43663,41600,1.193909,0.067328,0.609239,0.825453,-0.332038,-0.101522,-0.130782,-0.033697,0.424588,-0.291753,-0.350611,1.224264,1.168635,-0.413858,-0.286604,-0.717940,0.296345,-1.167928,-0.177968,-0.086323,-0.049950,0.213908,-0.068750,0.182281,0.557841,0.400000,NaN,NaN,NaN,NaN


In [5]:
#dataset information
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43664 entries, 0 to 43663
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    43664 non-null  int64  
 1   V1      43664 non-null  float64
 2   V2      43664 non-null  float64
 3   V3      43664 non-null  float64
 4   V4      43664 non-null  float64
 5   V5      43664 non-null  float64
 6   V6      43664 non-null  float64
 7   V7      43664 non-null  float64
 8   V8      43664 non-null  float64
 9   V9      43664 non-null  float64
 10  V10     43664 non-null  float64
 11  V11     43664 non-null  float64
 12  V12     43664 non-null  float64
 13  V13     43664 non-null  float64
 14  V14     43664 non-null  float64
 15  V15     43664 non-null  float64
 16  V16     43664 non-null  float64
 17  V17     43664 non-null  float64
 18  V18     43664 non-null  float64
 19  V19     43664 non-null  float64
 20  V20     43664 non-null  float64
 21  V21     43664 non-null  float64
 22

In [6]:
#checking no of missing values in each column
credit_card_data.isnull().sum()

,0
Time,0
V1,0
V2,0
V3,0
V4,0
V5,0
V6,0
V7,0
V8,0
V9,0


In [7]:
#distribution of legit and fraud transaction
credit_card_data['Class'].value_counts()

,count
Class,
0.0,43529
1.0,134


This dataset is highly unbalanced


0-> Normal Transaction
1-> Fraud Transaction

In [8]:
#seperating the data for analysis
legit=credit_card_data[credit_card_data.Class == 0]
fraud=credit_card_data[credit_card_data.Class == 1]

In [9]:
print(legit.shape)
print(fraud.shape)

(43529, 31)
(134, 31)


In [10]:
#statistical measures of data
legit.Amount.describe()

,Amount
count,43529.000000
mean,90.302800
std,238.465356
min,0.000000
25%,7.580000
50%,24.900000
75%,81.400000
max,7879.420000


In [11]:
fraud.Amount.describe()

,Amount
count,134.000000
mean,93.928433
std,231.551248
min,0.000000
25%,1.000000
50%,6.455000
75%,99.990000
max,1809.680000


In [12]:
#comapre the values for both transaction
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,26892.929610,-0.210136,0.014404,0.735522,0.173494,-0.226584,0.106817,-0.092752,0.039222,0.185126,-0.047529,0.387051,-0.387485,0.207134,0.231837,0.106308,0.008212,0.159178,-0.085092,-0.031297,0.046501,-0.029740,-0.109589,-0.039132,0.010036,0.135999,0.022175,0.004776,0.004927,90.302800
1.0,25259.246269,-7.759814,5.619386,-10.672832,6.111351,-5.776140,-2.360976,-8.192938,4.040000,-3.571525,-7.553505,5.565765,-8.888617,0.432079,-9.037839,0.094471,-5.919782,-9.968009,-3.656270,0.871182,0.507150,0.931331,-0.270332,-0.315606,-0.096976,0.279383,0.168436,0.687503,0.063472,93.928433


Under-Sampling

Build a sample dataset containing similar distribution of normal transaction and fraud transaction

Number of fraud transaction -> 134

In [13]:
legit_sample=legit.sample(n=134)

Concatinating two dataframes

In [14]:
new_dataset=pd.concat([legit_sample, fraud], axis=0)

In [15]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
21156,31507,-1.404774,-0.617184,1.372177,-1.908577,0.444022,0.107206,0.559978,-0.184044,1.738977,-0.547097,1.577207,1.001562,-0.573405,-0.466443,0.335447,-1.001996,-0.328353,-0.144994,-0.465933,-0.297713,0.170331,1.462591,0.371061,-0.258045,-0.386731,-0.791574,-0.172522,-0.065879,42.60,0.0
5528,5604,-0.246704,0.817282,0.045696,0.059680,2.919028,3.570344,0.220610,0.501822,0.658052,-0.129014,0.784407,-2.926092,1.601394,1.699377,0.703504,-0.240147,0.059068,0.829775,0.926533,0.171160,-0.091664,0.020739,-0.258796,0.952965,-0.081649,-0.330058,-0.153193,-0.074494,7.50,0.0
38650,39482,1.153577,-0.993790,0.402143,-0.559176,-1.093150,-0.201451,-0.617205,0.019077,-0.606840,0.597683,0.800005,0.199075,-0.080921,-0.287679,-1.065089,0.987351,0.283797,-1.138250,1.322948,0.284058,0.006388,-0.193223,-0.116982,0.024147,0.410662,-0.341125,-0.004850,0.022743,119.00,0.0
6624,8130,-9.175447,7.906022,-2.068334,-0.901683,-2.178911,4.675277,-8.314425,-22.605711,3.095693,2.241447,1.979352,0.054747,0.472042,2.935415,-0.804331,0.361195,2.512373,0.468013,-0.837287,-4.566619,22.579714,-8.483441,3.638520,-0.533970,0.428590,0.222205,1.159270,0.179528,23.98,0.0
21993,31977,-0.711168,1.430149,1.076119,0.175901,-0.393721,-1.200677,0.425501,0.291110,-0.695890,-0.760635,0.055021,0.569551,0.840914,-0.236126,0.752442,0.376592,0.254171,-0.293042,-0.224889,0.051287,-0.188627,-0.581789,0.084464,0.678510,-0.153503,0.051539,0.123658,0.040569,8.99,0.0


In [16]:
new_dataset.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
43061,41353,-15.020981,8.075240,-16.298091,5.664820,-11.918153,-4.246957,-14.716668,9.435084,-6.795398,-15.124163,6.037666,-11.676722,1.326979,-12.381606,-0.071639,-9.484143,-19.671881,-7.643273,1.169064,-0.995787,2.525115,-0.832074,-0.186117,0.429781,0.697103,0.056031,-1.310888,-0.707403,34.12,1.0
43160,41397,-14.970346,8.401421,-16.867238,8.252334,-13.565130,-2.782438,-14.263735,9.643419,-7.701499,-14.226698,5.610999,-11.793979,1.598387,-11.606970,-0.044180,-9.286955,-19.899726,-7.518051,1.243517,-1.614660,2.714045,-0.101355,-0.439666,0.519514,0.789328,0.064357,-1.621386,-1.104819,273.01,1.0
43204,41413,-15.140450,7.378042,-16.356367,9.194935,-13.466163,-2.958431,-16.165539,10.075254,-7.901821,-13.009403,4.828235,-11.058401,1.998126,-8.592047,0.540385,-10.615745,-21.017898,-8.397393,3.183559,-1.691482,3.147428,0.341678,-1.150162,0.795190,-0.194542,0.145964,-2.458680,-1.189888,106.55,1.0
43428,41505,-16.526507,8.584972,-18.649853,9.505594,-13.793819,-2.832404,-16.701694,7.517344,-8.507059,-14.110184,5.299236,-10.834006,1.671120,-9.373859,0.360806,-9.899247,-19.236292,-8.398552,3.101735,-1.514923,1.190739,-1.127670,-2.358579,0.673461,-1.413700,-0.462762,-2.018575,-1.042804,364.19,1.0
43624,41582,-1.048005,1.300219,-0.180401,2.589843,-1.164794,0.031823,-2.175778,0.699072,-1.140208,-3.226787,3.490805,-2.454339,0.177189,-5.160608,-0.686114,-3.996145,-4.695563,-1.819161,1.461080,0.644993,0.549014,0.624321,-0.136663,0.131738,0.030921,-0.176701,0.504898,0.069882,39.45,1.0


In [17]:
new_dataset['Class'].value_counts()

,count
Class,
0.0,134
1.0,134


In [18]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,27718.328358,-0.078264,0.013547,0.739433,0.124672,-0.22581,0.171377,-0.195616,-0.289558,0.227612,0.017660,0.432729,-0.334134,0.032567,0.242412,0.171016,-0.065496,0.185409,-0.11677,-0.009594,0.091272,0.079212,-0.082428,-0.045301,0.071206,0.177336,-0.055556,0.018433,0.018366,90.984851
1.0,25259.246269,-7.759814,5.619386,-10.672832,6.111351,-5.77614,-2.360976,-8.192938,4.040000,-3.571525,-7.553505,5.565765,-8.888617,0.432079,-9.037839,0.094471,-5.919782,-9.968009,-3.65627,0.871182,0.507150,0.931331,-0.270332,-0.315606,-0.096976,0.279383,0.168436,0.687503,0.063472,93.928433


In [19]:
#Splitting the data into Features and Targets
X= new_dataset.drop(columns='Class', axis=1)
Y= new_dataset['Class']

In [20]:
print(X)

        Time         V1        V2         V3  ...       V26       V27       V28  Amount
21156  31507  -1.404774 -0.617184   1.372177  ... -0.791574 -0.172522 -0.065879   42.60
5528    5604  -0.246704  0.817282   0.045696  ... -0.330058 -0.153193 -0.074494    7.50
38650  39482   1.153577 -0.993790   0.402143  ... -0.341125 -0.004850  0.022743  119.00
6624    8130  -9.175447  7.906022  -2.068334  ...  0.222205  1.159270  0.179528   23.98
21993  31977  -0.711168  1.430149   1.076119  ...  0.051539  0.123658  0.040569    8.99
...      ...        ...       ...        ...  ...       ...       ...       ...     ...
43061  41353 -15.020981  8.075240 -16.298091  ...  0.056031 -1.310888 -0.707403   34.12
43160  41397 -14.970346  8.401421 -16.867238  ...  0.064357 -1.621386 -1.104819  273.01
43204  41413 -15.140450  7.378042 -16.356367  ...  0.145964 -2.458680 -1.189888  106.55
43428  41505 -16.526507  8.584972 -18.649853  ... -0.462762 -2.018575 -1.042804  364.19
43624  41582  -1.048005  1.30021

In [21]:
print(Y)

21156    0.0
5528     0.0
38650    0.0
6624     0.0
21993    0.0
        ... 
43061    1.0
43160    1.0
43204    1.0
43428    1.0
43624    1.0
Name: Class, Length: 268, dtype: float64


To split data into training data and test data

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)


In [23]:
print(X.shape, X_train.shape, X_test.shape)

(268, 30) (214, 30) (54, 30)


Model Training

Logistic Regression

In [38]:
model=LogisticRegression(max_iter=1000)

In [39]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

In [40]:
X_train_prediction= model.predict(X_train)
training_accuracy=accuracy_score(X_train_prediction, Y_train)

In [41]:
print('Accuracy on training data :' , training_accuracy)

Accuracy on training data : 0.9813084112149533


In [42]:
X_test_prediction= model.predict(X_test)
test_accuracy=accuracy_score(X_test_prediction, Y_test)

In [43]:
print('Accuracy on test data:', test_accuracy)

Accuracy on test data: 0.9814814814814815
